In [1]:
# 라이브러리
import pandas as pd
import numpy as np
import joblib

In [2]:
# 미리 만든 모델 불러오기
PATH = './'

genre_svd = joblib.load(PATH + 'genre_svd.joblib.gzip')
country_svd = joblib.load(PATH + 'country_svd.sav.joblib.gzip')
people_svd = joblib.load(PATH + 'people_svd.joblib.gzip')

In [3]:
# 데이터 파일 불러오기
from kmr_dataset import load_rates
from kmr_dataset import get_paths

paths = get_paths(size='5m')
# paths = get_paths(size='2m')
# rates, timestamps = load_rates(size='small')
rates, timestamps = load_rates(size='5m')

skip 87439 lines which are duplicated (user, item), #uniques=4943613


In [4]:
castings, countries, genres, movies, peoples, rates = [pd.read_csv(path, delimiter = delim) for path, delim in zip(paths, [None] * 3 +['\t'] * 2 + [None])]

In [5]:
# 장르, 배우, 나라 index 저장
genre_index = genres.genre.unique()
people_index = peoples.people.unique()
country_index = countries.country.unique()

In [6]:
# 하나의 영화가 동시에 여러 속성을 가지는 경우가 있어 list로 합침
genres = genres.groupby("movie")["genre"].apply(list)
countries = countries.groupby("movie")["country"].apply(list)
# 조연배우까지 모두 넣었을때 데이터가 희석되어 주연배우만 계산(모델과 동일)
castings = castings[castings.leading == 1]
castings = castings.groupby("movie").agg({"people" : lambda x : list(x), "order" : lambda x : list(x), "leading" : lambda x : list(x)})

In [8]:
df = pd.merge(left = movies[["movie","title","title_eng"]], right = genres, how = "inner", on = "movie")
df = pd.merge(left = df , right = countries, how = "inner", on = "movie")
df = pd.merge(left = df , right = castings[["people"]], how = "inner", on = "movie")

df.head()

,movie,title,title_eng,genre,country,people
0,10001,시네마 천국,"Cinema Paradiso , 1988","[드라마, 멜로/로맨스]","[이탈리아, 프랑스]","[4374, 178, 3241, 47952]"
1,10002,빽 투 더 퓨쳐,"Back To The Future , 1985","[SF, 코미디]",[미국],"[1076, 4603, 917, 8637, 5104]"
2,10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989","[SF, 코미디]",[미국],"[1076, 4603]"
3,10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990","[서부, SF, 판타지, 코미디]",[미국],"[1076, 4603, 1031, 5104]"
4,10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977","[판타지, 모험, SF, 액션]",[미국],"[1007, 535, 215]"


In [9]:
# 특정 유저(uid)가 각각의 장르, 배우, 나라에 대해 얼만큼 점수를 주었는지 결과를 딕셔너리로 반환한다
def rating_dict(user_id):
    ugr = {genre : genre_svd.predict(user_id, genre).est for genre in genre_index}
#     ugr = dict(sorted(ugr.items(), reverse=True, key=lambda item: item[1]))
    upr = {people : people_svd.predict(user_id, people).est for people in people_index}
#     upr = dict(sorted(upr.items(), reverse=True, key=lambda item: item[1]))
    ucr = {country : country_svd.predict(user_id, country).est for country in country_index}
#     ucr = dict(sorted(ucr.items(), reverse=True, key=lambda item: item[1]))
    return ugr, upr, ucr

In [12]:
# 특정 유저(uid)가 선호하는 속성에 맞는 영화를 내림차순으로 나열한다
# w1 : 장르 선호도, w2 : 나라 선호도, w3 : 배우 선호도
# w1, w2, w3는 합이 1이 되도록 하는것을 추천한다

def top_rate_movie(uid, w1, w2, w3):
    ugr, upr, ucr = rating_dict(uid)
    g_rate, c_rate, p_rate = [], [], []
    for index in df.index:
        g_rate.append(np.mean([ugr[gen] for gen in df.loc[index].genre]))
        c_rate.append(np.mean([ucr[con] for con in df.loc[index].country]))
        p_rate.append(np.mean([upr[peo] for peo in df.loc[index].people]))
    t_rate = [w1 * g_rate[i] + w2 * c_rate[i] + w3 * p_rate[i] for i in range(len(g_rate))]
#     영화 제목과 환산 점수 dictionary
    t_rate = {df.loc[df.index[i]].title : t_rate[i] for i in range(len(t_rate))}
#     점수 순으로 정렬
    t_rate = dict(sorted(t_rate.items(), reverse=True, key=lambda item: item[1]))
    return t_rate

In [13]:
user_id = 4326
top_rate_movie(user_id, .5, .2, .3)

{'마징가 Z': 8.960076262122634,
 '마징카이저': 8.952325116471116,
 '우주전함 야마토 - 시리즈 2': 8.928997397725357,
 '우주전함 야마토 - 시리즈 3': 8.928997397725357,
 '마징가 Z 대 암흑대장군': 8.888451548432448,
 '마징가 Z 대 데빌맨': 8.888451548432448,
 '우주전함 야마토 - 새로운 여행': 8.887674449884354,
 '전설거신 이데온 극장판 - 발동편': 8.883948301546077,
 '미소녀 전사 세일러 문 S': 8.881008762483646,
 '기동전사 건담 제08 MS소대': 8.85221103648246,
 '윈드랜드': 8.85091955890743,
 '도라에몽 - 노비타와 로봇 왕국': 8.848337787614076,
 '도라에몽 - 노비타의 태양왕전설': 8.848337787614076,
 '극장판 도라에몽 진구의 바다 대모험': 8.848337787614076,
 '도라에몽 - 노비타와 꿈의 삼총사': 8.848337787614076,
 '도라에몽 - 노비타와 구름 왕국': 8.848337787614076,
 '도라에몽 - 노비타의 도라비안 나이트': 8.848337787614076,
 '도라에몽 - 노비타와 애니멀 혹성': 8.848337787614076,
 '도라에몽 - 노비타의 일본탄생': 8.848337787614076,
 '도라에몽 - 노비타의 패러랠 서유기': 8.848337787614076,
 '도라에몽 - 노비타와 용의 기사': 8.848337787614076,
 '도라에몽 - 노비타와 철인병단': 8.848337787614076,
 '도라에몽 - 노비타의 우주소전쟁': 8.848337787614076,
 '도라에몽 - 노비타의 마계대모험': 8.848337787614076,
 '도라에몽 - 노비타의 해저귀암성': 8.848337787614076,
 '신데렐라 더 캣': 8.8399401